# Import Library

In [1]:
import pandas as pd, numpy as np
from google.colab import drive
# Mount Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Melihat 5 data pertama

In [2]:
df = pd.read_csv('/content/gdrive/My Drive/DigitalSkola/Dataset/Breast_cancer.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


# Menginilisalisasi data menjadi X dan y kemudian menghapus data yang tidak di pakai.

In [3]:
X = df.drop(['diagnosis','id','Unnamed: 32'],1)
y = df['diagnosis']

# Membagi data menjadi data test dan data train

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.3,random_state = 123)

# Membuat function untuk evaluasi model

In [5]:
def evaluasi_model(model,X_test,y_test):
    from sklearn.metrics import accuracy_score
    y_pred = model.predict(X_test)
    return accuracy_score(y_test,y_pred)

# Model yang pertama di coba ada SVM

In [30]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

# Melihat Score dari SVM

In [31]:
evaluasi_model(svm,X_test,y_test) ## Murni Svm

0.9005847953216374

In [33]:
from sklearn.metrics import accuracy_score
y_train_pred = svm.predict(X_train)
accuracy_score(y_train, y_train_pred)

0.907035175879397

# Membuat Tunning untuk model SVM

In [8]:
params = {'C':[0.5,1,2],'kernel':['linear','rbf']}

In [9]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=svm,
             param_grid=params,
             scoring = 'accuracy',
             n_jobs = 2,
             cv = 3
            )

In [10]:
grid.fit(X_train,y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=2,
             param_grid={'C': [0.5, 1, 2], 'kernel': ['linear', 'rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

# Berikut Score yang di dapat setelah di lakukan Tuning

In [11]:
evaluasi_model(grid,X_test,y_test) ## Svm Tunning

0.9824561403508771

# Mencoba dengan model kedua yaitu Voting

In [12]:
from sklearn.ensemble import VotingClassifier 

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

logreg = LogisticRegression()
dtc = DecisionTreeClassifier()
svm = SVC()
knn = KNeighborsClassifier()

list_model = [('lr',logreg),('tree',dtc),('svm',svm),('knn',knn)]

In [14]:
vote = VotingClassifier(list_model)
vote.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('tree',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini...
                                  decisi

# Berikut Score yang di dapat lebih rendah dari svm yang telah di tunning

In [15]:
evaluasi_model(vote,X_test,y_test)

0.9298245614035088

# Model yang ketiga mencoba dengan model Bagging

In [16]:
from sklearn.ensemble import BaggingClassifier

In [17]:
bagging = BaggingClassifier()
bagging.fit(X_train,y_train)

BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=10,
                  n_jobs=None, oob_score=False, random_state=None, verbose=0,
                  warm_start=False)

# Score yang di dapat hampir mendekati hasil SVM yang telah di tunning.

In [18]:
evaluasi_model(bagging,X_test,y_test)

0.9883040935672515

# Model yang keempat menggunakan Random Forest

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
rf = RandomForestClassifier(random_state=444)
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=444,
                       verbose=0, warm_start=False)

# Score yang di dapat sama dengan SVM yang telah di tunning

In [21]:
evaluasi_model(rf,X_test,y_test)

0.9824561403508771

# Model yang kelima menggunakan Gradient Boosting

In [22]:
from sklearn.ensemble import GradientBoostingClassifier

In [23]:
grad = GradientBoostingClassifier()
grad.fit(X_train,y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

# Score yang di dapat sama dengan SVM yang udah di tunning dan Random Forest

In [24]:
evaluasi_model(bagging,X_test,y_test)

0.9883040935672515

# Melakukan tunning pada Random Forest

In [25]:
params = {'n_estimators':[50,100,150],
          'max_features':['auto','sqrt','log2']}

In [26]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=rf,
             param_grid=params,
             scoring = 'accuracy',
             n_jobs = 2,
             cv = 3
            )

In [27]:
grid.fit(X_train,y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=444,
                                 

# Score yang di dapat.

In [28]:
evaluasi_model(grid,X_test,y_test)

0.9824561403508771